In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

>>> --------- 🔁 What "Churn" Means – Real-World Examples by Industry ------------

> 📱 Telecom Company

- Churn = Customer cancels their mobile/internet plan or switches providers.

> 🎵 Streaming Service (e.g., Netflix, Spotify)

- Churn = Subscriber cancels their subscription or stops renewing it.

> 🛍️ E-commerce Website

- Churn = Regular customer stops making purchases for an extended period.

> 🏋️ Gym or Fitness Center

- Churn = Member cancels or doesn’t renew their membership.

> ☁️ SaaS (Software-as-a-Service)

- Churn = User cancels their subscription to a software/platform.

> 🏦 Banking or Financial Services

- Churn = Customer closes their account or switches to another bank.

> 🎓 Online Learning Platform

- Churn = Student stops enrolling in courses or cancels their membership.

> 🚗 Ride-Sharing App (e.g., Uber, Lyft)

- Churn = User stops using the app or switches to a competitor.

> Predictive Modeling Objective

- Develop a classification model to predict whether a customer will stay or churn based on historical customer data.

In [2]:
## using bank dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocessing the data
data=data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Encode categorical variables
label_encoder_gender = LabelEncoder()
data["Gender"]= label_encoder_gender.fit_transform(data["Gender"])

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## ONe-hot encoding for Geography
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
## combine one-hot encoded columns with the original dataframe
data = pd.concat([data, geo_encoded_df], axis=1)
data = data.drop(['Geography'], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## save the encoder and the scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

In [11]:
## Divide the data into features and target variable
X = data.drop(['Exited'], axis=1)
y = data['Exited']

## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


    

In [12]:
## Save the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [13]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


## ANN Implementation

> Sequential Model : 
* It mean that nodes will be inter-connect.
* A Sequential model means the data flows in one direction, from input → through each layer → to output, step by step.
- We do forward and backward propagation which is one sequential 
- if we have 2 inputs which is connect with 3 nodes of hidden layer(H1) so 3*2=6 so 6 weights is required and 3 bias. Total 6+3+1=9 parameters are trainable which will be using in forward and backward propagation.
- Bias will also be added to output

> Steps for Creating ANN : 
- Initializing a sequential network.
- To create Hidden neuron we will be using Dense e.g Dense(64) so 64 neuron will be created in the hidden layer 
- We will be using Activation Function (sigmoid, ReLU, Leaky ReLU etc).
- For o/p use sigmoid, softmax and for hidden layer use ReLU
- Optimizer : Useful for backpropagation --> updating the weights
- Loss Function (To minimum loss, we'll be using gradient descent)
- Matrix ---> for accuracy

- For Training information --> storing the logs into some folder which able to use TensorBoard library --> which is used to display these logs in way to understand it using visualizations

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

import datetime

In [15]:
X_train.shape

(8000, 12)

> For Sequential :
- To check number of columns which is input for the model : `X_train.shape[1]`
- `Dense` is used for hidden layers.
- When you are creating Dense you have to give input shape like how many inputs are coming from X_train data.

In [16]:
## Build the ANN model

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## For first hidden layer(connected with input layer), specify input shape is mandatory. 
    #Dropout(0.2),
    Dense(32, activation='relu'),   ## Second hidden layer
    #Dropout(0.2),
    
    Dense(1, activation='sigmoid')  ## Output layer with sigmoid activation for binary classification
])

d:\0 ML BootCamp\52_End_to_End_Deep_Learning_Project_Using_ANN\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<img src="General-scheme-of-an-artificial-neural-network-ANN.png" alt="ANN Diagram" width="500" height="450"/>


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

> Param # : 
- It is the combination of Weights and bias

In [18]:
import tensorflow 


## It is because adam optimizer have fixed learning rate, therefore we have changed the learning rate to 0.001
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

loss= tensorflow.keras.losses.BinaryCrossentropy()


In [19]:
## Forward and basckward propagation
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

- `logs/fit/` is used to make this folder.
- `datetime.datatime.now()` used datetime format which help us to see this 
- `strftime("%Y%m%d-%H%M%S")` is used to capture logs

In [20]:
## Set up for training using TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

- TensorBoard is used to visualize the log_dir directory

In [21]:
tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

`EarlyStopping` is used if the loss value is not further minimizing it will stop the training

In [22]:
## setup the EarlyStopping callback
## patence is the number of epochs with no improvement after which training will be stopped.
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [23]:
## Train the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=100, 
                    #batch_size=32, 
                    #validation_split=0.2, 
                    callbacks=[early_stopping, tensorflow_callbacks]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8102 - loss: 0.4387 - val_accuracy: 0.8505 - val_loss: 0.3676
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8490 - loss: 0.3635 - val_accuracy: 0.8565 - val_loss: 0.3565
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8580 - loss: 0.3475 - val_accuracy: 0.8610 - val_loss: 0.3450
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8571 - loss: 0.3475 - val_accuracy: 0.8620 - val_loss: 0.3462
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8622 - loss: 0.3335 - val_accuracy: 0.8655 - val_loss: 0.3403
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8637 - loss: 0.3310 - val_accuracy: 0.8605 - val_loss: 0.3479
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8637 - loss: 0.3447 - val_accuracy: 0.8645 - val_loss: 0.3351
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8596 - loss: 0.3385 - v

In [24]:
## Save the model
model.save('churn_model.h5')

In [25]:
## Let's load the TensorBoard Extension
%load_ext tensorboard

In [26]:
## Start TensorBoard
%tensorboard --logdir logs/fit/20250525-153601

Reusing TensorBoard on port 6007 (pid 15088), started 13:33:49 ago. (Use '!kill 15088' to kill it.)

<iframe id="tensorboard-frame-3655c97c0872a542" width="100%" height="800" frameborder="0">
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById("tensorboard-frame-3655c97c0872a542");
          const url = new URL("/", window.location);
          const port = 6006;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>